
## Lectura del set de datos
Así que crearemos inicialmente una función que nos permita cargar el set de imágenes de entrenamiento y validación, junto con la categoría a la que pertenece cada una de estas imágenes:


In [ ]:
#En esta celda crearemos inicialmente una función que nos permita cargar el set de imágenes de entrenamiento y validación, junto con la categoría a la que pertenece cada una de estas imágenes:
import numpy as np
import os
import gzip

def load_set(ruta, tipo='train'):

  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)
    
  with gzip.open(ruta_categorias, 'rb') as rut_cat:
      etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
      imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas


#Acceso a google drive
Ahora sí vamos a importar la librería “drive” de Google Colab, que nos permitirá hacer la lectura de datos desde Google Drive. Una vez importada la librería, “montamos” nuestro “drive” en el servidor remoto de Google Colab y luego definimos la ruta completa en donde se encuentran almacenados los datos.

Estas son entonces las líneas de código que permiten acceder al “drive” y cargar el set de datos generando los respectivos sets de entrenamiento y validación:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ruta = '/content/drive/MyDrive/Fashion_MNIST'

X_train, Y_train = load_set(ruta, tipo='train')
X_test, Y_test = load_set(ruta, tipo='test')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Reajustar el tamaño de los datos
El set de datos debe contener una cantidad de datos que sea múltiplo de 128. Ya que para entrenar con la arquitectura de TPU, se necesita que el número de datos sea múltilo de 128, es decir, 128,256,512 y así sucesivamente. Es bueno mencionar, que esto no es necesario para entrenar con CPU o con GPU, pero la idea de este ejercicio es comparar los tiempos de entrenamiento, entonces, con este fin normalizaremos los valores de entrada al modelo en este paso. 

- El conjunto de datos de entrenamiento tiene 60000 datos, número más cercano a este, que es múltiplo de 128, es 59904.
-El conjunto de datos de entrenamiento tiene 10000 datos, número más cercano a este, que es múltiplo de 128, es 9984. 

**¿Preguntar como hacer el cálculo de los múltiplos?**

**Preguntar este parrafo: Finalmente, usamos “reshape” para reajustar el tamaño de los sets de entrenamiento y validación y así garantizar que cada dato es una imagen en escala de grises de 28 x 28 pixeles, que será la entrada a la red convolucional que queremos entrenar**

In [ ]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

#Importar tensorflow 2 (ya incluye Keras)

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

print('Versión de Tensorflow: ' + tf.__version__)

Versión de Tensorflow: 2.8.0


In [ ]:
##Definimos la semilla, para que el entrenamiento del modelo sea comparable, cuando se hace por CPU, GPU y TPU. 
tf.random.set_seed(200)

#Se crea el contenedor del modelo con el módulo secuencial 
model = tf.keras.models.Sequential()
#Se definen las tres capas convolucionales

#Primera capa convolucional (Filtros 64),, primero se hace una normalización sobre los datos, para facilitar el proceso de convergencia, posteriormente se agregan los filtros
#, a continuación se agrega la capa de max pooling, para reducir las dimensiones y por último se agrega una capa de dropout para evitar el sobre-ajuste. 
#esto se repite para las tres capas convolucionales. 

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

#Segunda capa convolucional (Filtros 128)
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

#Tercera capa convolucional (Filtros 256)
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

#Se aplanan los datos para convertirlos en vectores con el módulo Flatten. 
model.add(tf.keras.layers.Flatten())
#Se agrega una red neuronal con 256 neuronas
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
#Se declara la salida de la red neuronal con 10 neuronas y con la función de activación softmax, para realizar el proceso de clasificación de las imágenes. 
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

#Se define el optimizador, la función de error y la métrica de desempeño. 
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_9 (Batc  (None, 28, 28, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_9 (Conv2D)           (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_10 (Bat  (None, 14, 14, 64)       256       
 chNormalization)                                                
                                                      

#Generar checkpoint

In [ ]:
#Generar la dirección donde se guardan los datos
checkpoint_path="/content/drive/MyDrive/Introduccion_Inteligencia_Artificial/TalleresyTareas/PrimeraTarea/CPU"
checkpoint_dir=os.path.dirname(checkpoint_path)

#Crear una llamada que salva los modelos
cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=False,verbose=1)

# Entrenamiento con CPU

In [ ]:
import timeit

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1,callbacks=[cp_callback])
  
  return None

cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')

Epoch 1/2
468/468 [==============================] - ETA: 0s - loss: 0.0750 - accuracy: 0.9799
Epoch 1: saving model to /content/drive/MyDrive/Introduccion_Inteligencia_Artificial/TalleresyTareas/PrimeraTarea/CPU
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Introduccion_Inteligencia_Artificial/TalleresyTareas/PrimeraTarea/CPU/assets
468/468 [==============================] - 549s 1s/step - loss: 0.0750 - accuracy: 0.9799 - val_loss: 0.0600 - val_accuracy: 0.9838
Epoch 2/2
468/468 [==============================] - ETA: 0s - loss: 0.0642 - accuracy: 0.9823
Epoch 2: saving model to /content/drive/MyDrive/Introduccion_Inteligencia_Artificial/TalleresyTareas/PrimeraTarea/CPU
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Introduccion_Inteligencia_Artificial/TalleresyTareas/PrimeraTarea/CPU/assets
468/468 [==============================] - 550s 1s/step - loss: 0.0642 - accuracy: 0.9823 - val_loss: 0.0327 - val_accuracy: 0.9919


In [ ]:
print('Tiempo de entrenamiento ' + str(cpu_time)+ ' segundos')

Tiempo de entrenamiento 1131.0402922320009 segundos
